# Prelude



[**Resonators**](https://github.com/0xmaddie/0xmaddie/blob/main/resonators)
is a Python library that uses data races and particle swarm dynamics
to approximate functions. You can train a resonator like a transformer
but it uses Python objects instead of vectors.

## Particle swarms and oracle machines
How does "swarm intelligence" work? A logical dependence on an
implicit function representation observed by repeated measurement of
collective behavior is a type of non-algorithmic computation because
there's no effective procedure that would give you that information.

You can use a particle swarm to train something called a component
system — like a Turing machine or a bunch of Python or Lisp functions
— in a manner similar to a neural net. The idea is to use the
priorities of racing threads as an implicit representation of an
energy function learned from a dataset.

The important part is that each individual thread operation can only
use data that is averaged across every thread. This provides an
analogy to continuity and allows you to pass to the mean field limit,
so you can optimize in a manner similar to a neural net.

Imagine a class of programs written as lists of rules of the following
form: If the state has a certain property, transform the state in a
certain way. To execute these programs, evaluate the conditions in the
order they are written until one is true, and then apply the
corresponding transformation.

We'd like to add weights to the space of all possible programs of this
shape and tune them in alignment with a dataset. In other words, we'd
like to create an oracle machine. The resonator is a machine of this
type that uses an analogy to continuous functions through particle
swarm dynamics to explore this space of programs.

The resonator contains lists of every possible property and
transformation, respectively called the "inputs" and the
"outputs". When a layer is presented with a swarm of states, the
inputs race according to their advice. The advice is provided in the
form of the bias of a coin; on each step of the race the coins
represent the probability that the respective input will be next in
line until the inputs are sorted.

Then, in the sorted order, the inputs attempt to provide a measurement
of the swarm states that satisfy some property. If an input is unable
to provide the minimum number of states required for a measurement,
then the search proceeds with the next input in line. If no inputs are
able to provide a measurement, the procedure raises an exception.

If the inputs are able to provide a measurement, then the outputs
condition on the winner of the input race and perform their own race
in the same manner. The winner of the output race then combines the
entire measurement of states into a single state in a manner similar
to a crossover operation from genetic programming. It's important that
the output's final state depends on many states sampled from the swarm
at once in order to provide the analogy to continuity.

This input-output process is repeated until a measurement of states
has been provided by the outputs. Then, this measurement races with
the initial states according to the advice as a type of residual
connection. The idea is that the probabilities of all of these events
are an implicit representation of a function learned from a
dataset. In this sense, the coins are an oracle provided to a machine
allowing it to self-organize.


In [ ]:
import google
google.colab.drive.mount('/content/drive')
!cp /content/drive/MyDrive/secret.py .
import secret

In [ ]:
!pip install bitsandbytes accelerate

In [ ]:
import huggingface_hub
huggingface_hub.login()

# Construction

## Decoder

In [ ]:
import numpy as np

class Decoder:
  def __init__(self, inputs, outputs, weights):
    self.inputs  = inputs
    self.outputs = outputs
    self.weights = weights

  @property
  def density(self):
    return len(self.inputs)*len(self.outputs)

  # The number of weights per layer.
  @property
  def layer_capacity(self):
    return len(self.inputs) + len(self.inputs) * len(self.outputs) + 1

  @property
  def layers(self):
    return len(self.weights)/self.layer_capacity

  @property
  def capacity(self):
    return len(self.weights)

  # The weights associated with the inputs for a certain layer.
  def win(self, layer):
    lhs = layer * self.capacity
    return self.weights[lhs:lhs+len(self.inputs)]

  # The weights associated with the outputs for a certain layer.
  def wout(self, layer, input_index):
    input_lhs  = layer * self.capacity
    output_lhs = input_lhs + len(self.inputs)
    target_lhs = output_lhs + input_index * len(self.outputs)
    return self.weights[target_lhs:target_lhs+len(self.outputs)]

  # The weights associated with the residual for a certain layer.
  def wres(self, layer):
    input_lhs    = layer * self.capacity
    residual_idx = input_lhs + len(self.inputs) + len(self.inputs) * len(self.outputs)
    return self.weights[residual_idx]

  def sort(self, components, weights):
    # Each weight is the probability that the corresponding
    # component will be next in line.
    indices = np.random.choice(
      len(inputs),
      size=len(inputs),
      replace=False,
      # TODO: softmax the weights here to make
      # sure they always sum to 1
      p=weights,
    )
    return indices

  def __call__(self, swarm):
    for time in range(self.layers):
      inputw    = self.win(time)
      residualw = self.wres(time)
      hidden    = []
      for i in range(self.purity):
        # The inputs race to provide a measurement.
        inputbest = None
        for inputid in self.sort(self.inputs, inputw):
          input  = self.inputs[inputid]
          sample = []
          for state in swarm:
            if input(state):
              sample.append(state)
            if len(sample) >= threshold:
              inputbest = inputid
              break
        # If no inputs were able to provide a large enough measurement
        # then an exception is raised.
        if len(sample) < threshold:
          raise ValueError()
        activity = False
        # The outputs condition on the winner of the race
        # and perform their own race to provide a state.
        outputw  = self.wout(time, inputbest)
        for outputid in self.sort(self.outputs, outputw):
          output = self.outputs[outputid]
          try:
            # The output is applied to *all* of the sample values.
            # It's important to mix states across the swarm to allow
            # the analogy to continuity.
            point    = output(sample)
            activity = True
            hidden.append(point)
            break
          except OutputError:
            pass
        if not activity:
          raise ValueError()
      # The final states race with the initial states.
      target = []
      for i in range(self.purity):
        if np.random.random() < residualw:
          state = np.random.choice(hidden)
        else:
          state = np.random.choice(swarm)
        target.append(state)
      swarm = target
    return swarm

In [ ]:
import transformers
import torch

class Llama:
  tokenizer: transformers.AutoTokenizer
  model: transformers.AutoModelForCausalLM
  system_prompt: str
  temperature: float
  max_new_tokens: int
  terminators: list
  purity: int
  quota: int

  def __init__(
    self,
    system_prompt: str = 'You are a helpful assistant.',
    temperature: float = 0.7,
    max_new_tokens: int = 4096,
    purity: int = 100,
    quota: int = 1000,
  ):
    model_name     = "meta-llama/Meta-Llama-3-8B-Instruct"
    self.tokenizer = transformers.AutoTokenizer.from_pretrained(
      model_name,
      #cache_dir=secret.home,
      #local_files_only=True,
    )
    self.model     = transformers.AutoModelForCausalLM.from_pretrained(
      model_name,
      device_map="auto",
      load_in_8bit=True,
      #cache_dir=secret.home,
      #local_files_only=True,
    )
    self.terminators = [
      self.tokenizer.eos_token_id,
      self.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    self.system_prompt  = system_prompt
    self.temperature    = temperature
    self.max_new_tokens = max_new_tokens
    self.purity         = purity
    self.quota          = quota

  def __apply_chat_template(self, state):
    prompt = [{ 'role': 'system', 'content': self.system_prompt }]
    for index, content in enumerate(state):
      if index%2 == 0:
        role = 'user'
      else:
        role = 'assistant'
      prompt.append({ 'role': role, 'content': content })
    print(f'<prompt>{prompt}</prompt>')
    input_ids = self.tokenizer.apply_chat_template(
      prompt,
      #tokenize=True,
      add_generation_prompt=True,
      return_tensors='pt',
    ).to(self.model.device)

    return input_ids

  def get(self, state):
    source_ids = self.__apply_chat_template(state)
    targets = self.model.generate(
      source_ids,
      max_new_tokens=self.max_new_tokens,
      do_sample=True,
      temperature=self.temperature,
      eos_token_id=self.terminators,
    )
    target_ids = targets[0][source_ids.shape[-1]:]
    target = self.tokenizer.decode(target_ids, skip_special_tokens=True)
    return target

  def reduce(self, input, map, reduce):
    quota = self.quota
    samples = []
    while quota > 0 and len(samples) < self.purity:
      quota -= 1
      try:
        object = map(self.get(input))
        samples.append(object)
      except ValueError:
        pass
    if len(samples) < self.purity and quota == 0:
      raise ValueError(f'quota consumed')
    output = reduce(samples)
    return output

In [ ]:
llm = Llama(
  system_prompt=f'''
You are a helpful assistant.
'''.strip(),
  temperature=0.7,
  max_new_tokens=1024,
  purity=1,
  quota=3,
)

In [ ]:
from bs4 import BeautifulSoup

def get_tag_body(document, tag_name):
  # Perform string replacement to transform <value> tags into <div class="value">
  document = document.replace(f"<{tag_name}>", f'<div class="{tag_name}">').replace(f"</{tag_name}>", "</div>")
  # Parse the modified HTML document with BeautifulSoup
  soup = BeautifulSoup(document, 'html.parser')
  # Find all divs with the class "value"
  value_divs = soup.find_all('div', class_=tag_name)
  # Ensure there is exactly one div with class "value"
  if len(value_divs) != 1:
    raise ValueError(f"Expected exactly one <div class='{tag_name}'> tag, but found {len(value_divs)}.")
  # Return the text content of the <div class="value">
  return value_divs[0].get_text()

class Repository:
  llm: Llama

  def __init__(self, llm):
    self.llm = llm

  def input(self, predicate):
    def map_input(x):
      # get_tag_body uses BeautifulSoup to
      # 1. Replace <value> tags with <div class="value"> with string replacement.
      # 2. Ensure there is only one such tag, raising a ValueError if this is not true.
      # 3. Return the body of that tag as a string.
      print(f'checking bool in\n{x}')
      value = get_tag_body(x, 'value')
      print(f'value = {value}')
      return bool(value)
    def reduce_input(xs):
      numbers = [1 if bool(x) else 0 for x in xs]
      average = sum(numbers)/len(xs)
      print(f'average = {average}')
      return average > 0.5
    def body(state):
      prompt = f'''
You are analyzing prompts for a generative text-to-image model.
Your task is to determine whether this predicate:

```
{predicate}
```

is true of the following prompt:

```
{state}
```

First, provide a comprehensive, detailed, and nuanced analysis of
whether the predicate is true of the prompt. Then, place the word
True or the word False within <value> tags.
'''.strip()
      response = self.llm.reduce([prompt], map_input, reduce_input)
      return response
    return body

  def output(self, transform):
    def map_output(x):
      value = get_tag_body(x, 'value')
      return value
    def reduce_output(xs):
      return xs[0]
    def body(input, sample):
      prompt = f'''
You are creating prompts for a generative text-to-image model.
Your task is to crossover and mutate prompts that have a certain relation.

Consider the following text-to-image prompts:

```
{sample}
```

All of these prompts satisfy the following relation:

```
{input}
```

Your task is to both combine and mutate these prompts
according to the following principle:

```
{transform}
```

First, to perform your crossover operation, combine all of
the prompts in to a single prompt based on some theme you notice.
Then, determine how you will modify the prompt according to the
principle given to you. When you have produced a final prompt,
place it within <value> tags.
'''.strip()
      response = self.llm.reduce([prompt], map_output, reduce_output)
      return response
    return body

In [ ]:
repo = Repository(llm)

inputs = [
  repo.input('Is this a classical painting?'),
  repo.input('Is this an anime illustration?'),
  repo.input('Does this artwork belong to the Renaissance period?'),
  repo.input('Is this a modern abstract artwork?'),
  repo.input('Is this piece in the style of Impressionism?'),
  repo.input('Does this resemble a Baroque painting?'),
  repo.input('Is this artwork in the style of Cubism?'),
  repo.input('Is the artist associated with the Surrealism movement?'),
  repo.input('Does this image use techniques typical of watercolor painting?'),
  repo.input('Is this artwork a digital creation?'),
  repo.input('Does this represent a form of street art?'),
]

outputs = [
  repo.output('Make this an anime illustration.'),
  repo.output('Make this a classical painting.'),
  repo.output('Transform this image to reflect the Renaissance art style.'),
  repo.output('Create this image in a modern abstract style.'),
  repo.output('Convert this to an Impressionist style painting.'),
  repo.output('Reimagine this as a Baroque period painting.'),
  repo.output('Adapt this image to the Cubism art style.'),
  repo.output('Render this image in a Surrealist style.'),
  repo.output('Use watercolor techniques for this image.'),
  repo.output('Create this artwork using digital media.'),
  repo.output('Style this image as street art.'),
]

In [ ]:
is_classical = inputs[0]
fst = 'A digital art illustration by Ilya Kuvshinov.'
snd = 'An oil painting by John Singer Sargent'
print(f'{fst} = {is_classical(fst)}')
print(f'{snd} = {is_classical(snd)}')

In [ ]:
make_anime = outputs[0]
response = make_anime(
  'Is this a classical painting?',
  [
    'An oil painting of a woman holding a parasol by John Singer Sargent',
    'A still life of fruits on a table by Joaquin Sorolla',
    'A scene from Greek mythology by Leonardo da Vinci',
  ],
)

In [ ]:
print(response)

In [ ]:
num_weights = len(inputs)+len(inputs)*len(outputs)+1
decoder = Decoder(
  inputs=inputs,
  outputs=outputs,
  weights=np.random.rand(num_weights),
)
initial_state = ['a woman holding a parasol', 'an oil painting', 'by ilya kuvshinov']
final_states  = decoder(initial_state)
print(final)

## Tuning

In [ ]:
@dataclasses.dataclass(frozen=True)
class Optimizer:
  facc: float
  fres: float
  rate: float
  decay: float

  def __call__(self, pos, acc, grad):
    hacc  = self.interp(grad, acc, self.facc)
    res   = self.interp(grad, acc, self.fres)
    mix   = pos*self.decay+sign(res)
    delta = mix*self.rate
    return delta, hacc

class Tuner:
  quota: int
  purity: int
  density: int
  update: callable

  def measure(self, machine, data):
    energy = 0
    for i in range(self.purity):
      (init, eval)  = data()
      energy       += eval(machine(init))
    return energy/self.purity

  def __call__(self, init, cons, data):
    pos  = []
    acc  = []
    loss = []
    for _ in range(self.quota):
      for i in range(self.density):
        machine = cons(pos[i])
        loss[i] = self.measure(machine, data)
      center  = self.average(pos, loss)
      teacher = cons(center)
      cutoff  = self.measure(teacher, data)
      for i in range(self.density):
        if loss[i] <= cutoff:
          continue
        compass  = pos[i]-center
        sort     = compass*self.coherence
        search   = (compass@compass)*self.noise()
        gradient = sort+search
        vel, acc = self.update(pos[i], acc[i], gradient)
        pos[i]  += vel
        acc[i]   = acc
    return self.average(pos, loss)